In [1]:
import pandas as pd 
import numpy as np
from doepy import build
import seaborn as sns
import matplotlib.pyplot as plt
import pickle
import warnings
warnings.filterwarnings('ignore')

In [2]:
from IISEPaper import categorise
from IISEPaper import get_labeled_index
from IISEPaper import get_feature_label
from IISEPaper import get_labeled_set
from IISEPaper import get_training_Set
from IISEPaper import get_unlabeled_set
from IISEPaper import c_prediction
from sklearn.gaussian_process import GaussianProcessClassifier
from IISEPaper import predicted_region_divide
from IISEPaper import get_labeled_feasible_sample
from IISEPaper import get_feasible_rep
from IISEPaper import get_div_term
from IISEPaper import find_indices
from IISEPaper import get_select_index
from IISEPaper import GP_predict
from IISEPaper import labeled_fea_sample
from IISEPaper import remove_label
from IISEPaper import get_index_div

In [3]:
from Gpregression import GP_predict
from Gpregression import candidate_pool_ucb
from Gpregression import topQuery

In [4]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from IISEPaper import get_select_index

In [5]:
def labeled_fea_sample(x_labeled,y_labeled,y_r_labeled):
    index_0 = np.where(y_labeled == 0)[0]
    index_1 = np.where(y_labeled == 1)[0]
    return x_labeled[index_0], x_labeled[index_1],y_r_labeled[index_0]

In [6]:
def remove_label(full_list,remove_list):
    for n in remove_list:
        while n in full_list:
            full_list.remove(n)
            
    return full_list

In [7]:
def get_index_div(div, unlabeled_index):
    index=[]
    max_div = max(div)
    temp = list(div).index(max_div)
    index.append(unlabeled_index[temp])
    return index

In [9]:
data = pd.read_csv('dataset1104.csv')
ds,X_feature,y_ground,y_r = get_feature_label(data)
full_index = list(np.arange(len(data)))

In [11]:
data

,x1,x2,y,l
0,-1.0,-1.0,-0.756802,1
1,-0.9,-1.0,-0.746802,1
2,-0.8,-1.0,-0.716802,1
3,-0.7,-1.0,-0.666802,1
4,-0.6,-1.0,-0.596802,1
...,...,...,...,...
436,0.6,1.0,1.803198,1
437,0.7,1.0,2.133198,1
438,0.8,1.0,2.483198,1
439,0.9,1.0,2.853198,1


In [10]:
index_file = np.load('data2_5_index.npy',allow_pickle = True)

In [11]:
result=[]
acc=[]
for n in range(len(index_file)):
    labeled_index = list(index_file[n])
    full_index = list(np.arange(len(data)))
    x_labeled = X_feature[labeled_index]
    y_labeled = y_ground[labeled_index]
    y_r_labeled = y_r[labeled_index]
    unlabeled_index = remove_label(full_index,labeled_index)
    unlabeled_feature = X_feature[unlabeled_index]
    r_selected = y_r[labeled_index]
    labeled_feasible_sample, labeled_infeasible_sample,labeled_r = labeled_fea_sample(x_labeled,y_labeled,r_selected)
    r=[min(labeled_r)-1]
    f1=[]
    l=0
    
    # Change the termination criteria when test the objective optimization performance
    
    #while min(r) > 0 and len(r)<50:  
    
    while l<15:
        
        #set initial goast training set
        training_x = x_labeled
        training_y = r_selected.copy()
        y_pred_labeled_sudo,sigma_labeled_sudo = GP_predict(labeled_feasible_sample,labeled_r,labeled_infeasible_sample)
        training_y[np.where(y_labeled == 1)[0]]=y_pred_labeled_sudo
        
        #compute contrained ucb value using goast training set
        model = GaussianProcessClassifier()
        y_c_pred , y_prob , y_c_uncertainty = c_prediction(x_labeled, y_labeled, unlabeled_feature,model)

        
        y_c = y_ground.copy()
        y_c[unlabeled_index] = y_c_pred
        f1.append(accuracy_score(y_ground, y_c))

        y_pred_pool,sigma_pool = GP_predict(training_x,training_y,unlabeled_feature)
        ucb = -y_pred_pool+ 2*sigma_pool
        constrain_ucb = (1-y_c_pred) * (np.amax(y_prob, axis=1)) *ucb 
        
        #test 
        y_pred_pool.shape == y_c_pred.shape == constrain_ucb.shape
       
        #1select samples
        select_index_0 = get_index_div(constrain_ucb, unlabeled_index)
        max_0 = np.argmax(constrain_ucb)
        x_sudo = np.r_[training_x,X_feature[select_index_0]]
        y_r_sudo = np.append(training_y,y_pred_pool[max_0])
        labeled_index = labeled_index + select_index_0
        unlabeled_index = remove_label(full_index,labeled_index)
        unlabeled_feature = X_feature[unlabeled_index]
        y_pred_batch,sigma_bacth = GP_predict(x_sudo,y_r_sudo,unlabeled_feature)
        
        #2select samples
        select_index_1 = get_index_div(sigma_bacth, unlabeled_index)
        labeled_index = labeled_index + select_index_1
        unlabeled_index = remove_label(full_index,labeled_index)
        unlabeled_feature = X_feature[unlabeled_index]
        x_class_sudo =  np.r_[x_labeled,X_feature[select_index_0],X_feature[select_index_1]]
        y_class_sudo = np.append(y_labeled,[0,0])
        y_batch,y_batch_pro,y_batch_uncertainty = c_prediction(x_class_sudo, y_class_sudo, unlabeled_feature,model)
        
        #2select samples
        select_index_2 = get_index_div(y_batch_uncertainty, unlabeled_index)
        labeled_index = labeled_index + select_index_2
        x_labeled = X_feature[labeled_index]
        y_labeled = y_ground[labeled_index]
        y_r_labeled = y_r[labeled_index]
        unlabeled_index = remove_label(full_index,labeled_index)
        unlabeled_feature = X_feature[unlabeled_index]
        r_selected = y_r[labeled_index]
        
        labeled_feasible_sample, labeled_infeasible_sample,labeled_r = labeled_fea_sample(x_labeled,y_labeled,r_selected)
        r.append(min(labeled_r)-1)
        l=l+1
    
    result.append(r)
    acc.append(f1)
    print('Round {} finished'.format(n))

Round 0 finished
Round 1 finished
Round 2 finished
Round 3 finished
Round 4 finished
Round 5 finished
Round 6 finished
Round 7 finished
Round 8 finished
Round 9 finished
Round 10 finished
Round 11 finished
Round 12 finished
Round 13 finished
Round 14 finished
Round 15 finished
Round 16 finished
Round 17 finished
Round 18 finished
Round 19 finished
Round 20 finished


In [12]:
len(result)

21

In [ ]:
from itertools import zip_longest

temp = np.array(list(zip_longest(*result, fillvalue=0))).T

In [13]:
mean_al=np.array(acc).mean(axis=0)
std_al = np.array(acc).std(axis=0)

In [14]:
mean_al

array([0.86243386, 0.87431163, 0.88295001, 0.89018465, 0.89763524,
       0.90076666, 0.90422201, 0.90897311, 0.91394018, 0.91685563,
       0.91955512, 0.92247058, 0.9265738 , 0.92851744, 0.93283663])

In [15]:
std_al

array([0.04503362, 0.03262524, 0.03402063, 0.0269175 , 0.02813363,
       0.01940453, 0.01580197, 0.01555807, 0.01416058, 0.01298543,
       0.01114443, 0.01203187, 0.01122885, 0.01062709, 0.01142342])